In [11]:
import os
import mmcv
from mmcv import Config
from mmdet.datasets import (build_dataloader, build_dataset, replace_ImageToTensor) # 앞의 2개 --> mmdet/datasets/builder.py에 존재 / 마지막 1개 --> 
from mmdet.models import build_detector #mmdet/models/builder.py에 존재
from mmdet.apis import train_detector, single_gpu_test # mmdet/apis/train.py, test.py에 존재
from mmcv.runner import load_checkpoint
from mmcv.parallel import MMDataParallel
import pandas as pd
from pandas import DataFrame
from pycocotools.coco import COCO
import numpy as np
import json, wandb

## train part

In [14]:
classes = ("UNKNOWN", "General trash", "Paper", "Paper pack", "Metal", "Glass", 
           "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing")
# config file 들고오기
#cfg = Config.fromfile('../configs/faster_rcnn/faster_rcnn_r50_fpn_1x_coco.py')## faster_rcnn_hrnetv2p_w40_2x_coco
#파일 교체 시 여기 이름 바꾸기
config_dir_name = 'faster_rcnn'## hrnet
config_file_name = 'faster_rcnn_r50_fpn_1x_coco'
cfg = Config.fromfile('../configs/'+config_dir_name+'/'+config_file_name+'.py')

PREFIX = '/opt/ml/input/data/'
#working directory 이름 설정
#sub_working_dir_name = 'faster_rcnn_r50_fpn_1x_trash'
working_dir_name = '../work_dirs/' + config_file_name

# dataset 바꾸기
cfg.data.train.classes = classes
cfg.data.train.img_prefix = PREFIX
cfg.data.train.ann_file = PREFIX + 'train.json'
cfg.data.train.pipeline[2]['img_scale'] = (512, 512)

cfg.data.val.classes = classes
cfg.data.val.img_prefix = PREFIX
cfg.data.val.ann_file = PREFIX + 'val.json'
cfg.data.val.pipeline[1]['img_scale'] = (512, 512)

cfg.data.test.classes = classes
cfg.data.test.img_prefix = PREFIX
cfg.data.test.ann_file = PREFIX + 'test.json'
cfg.data.test.pipeline[1]['img_scale'] = (512, 512)

cfg.data.samples_per_gpu = 16    #batch size

cfg.seed=42 #42로 고정
cfg.gpu_ids = [0]
cfg.work_dir = working_dir_name

cfg.model.roi_head.bbox_head.num_classes = 11

cfg.optimizer_config.grad_clip = dict(max_norm=35, norm_type=2)

#wandb setting


## wandb setting

In [23]:
group_name = 'hrnet'; project_name = 'gjtak_od'; run_name = config_file_name
config_list = {
    'epoch' : cfg.runner.max_epochs,
    'batch_size' :  cfg.data.samples_per_gpu,
    'optimizer' : cfg.optimizer,
    'optimizer_config' : cfg.optimizer_config,
    'lr_config' : cfg.lr_config
}
cfg.log_config.hooks[1].init_kwargs['group']=group_name # group name(option)
cfg.log_config.hooks[1].init_kwargs['name'] = config_file_name # run name
cfg.log_config.hooks[1].init_kwargs['config'] = config_list # config
#cfg.log_config.hooks[1].init_kwargs

### config 저장

In [29]:
# config 객체 -> 파일(json)로 저장(저장 장소 = working directory와 같이)
#working directory 없으면 생성하기
try:
    if not os.path.exists(working_dir_name):
        os.makedirs(working_dir_name)
except OSError :
    print('Error: Creating directory :: ' + working_dir_name)

#cfg 객체 -> string(str이용) -> dict(eval이용) -> json 으로 저장
cfg_str = str(cfg)[str(cfg).find(')')+3:]
cfg_dict = eval(cfg_str)
with open(working_dir_name+'/config.json','w') as config_json_file :
    json.dump(cfg_dict, config_json_file, indent="\t")


In [28]:
#config 저장 확인
with open(working_dir_name+'/config.json', 'r') as f :
    json_data = json.load(f)
#json_data

In [30]:
model = build_detector(cfg.model)

2021-05-12 09:24:25,127 - mmdet - INFO - load model from: torchvision://resnet50
2021-05-12 09:24:25,128 - mmdet - INFO - Use load_from_torchvision loader
2021-05-12 09:24:25,356 - mmdet - WARNING - The model and loaded state dict do not match exactly

unexpected key in source state_dict: fc.weight, fc.bias



In [31]:
datasets = [build_dataset(cfg.data.train)]

loading annotations into memory...
Done (t=4.09s)
creating index...
index created!


In [33]:
#wandb watch
#wandb.watch(model, log='all')

In [34]:
train_detector(model, datasets[0], cfg, distributed=False, validate=True)

loading annotations into memory...
2021-05-12 09:24:57,682 - mmdet - INFO - Start running, host: root@7abc6b7bc976, work_dir: /opt/ml/p3-ims-obd-hansarang/gjtak_works/Object Detection/code/mmdetection_trash/work_dirs/faster_rcnn_r50_fpn_1x_coco
2021-05-12 09:24:57,682 - mmdet - INFO - workflow: [('train', 1)], max: 12 epochs
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
Done (t=0.89s)
creating index...
index created!
wandb: Currently logged in as: pstage12 (use `wandb login --relogin` to force relogin)


019, memory: 6896, loss_rpn_cls: 0.0329, loss_rpn_bbox: 0.0435, loss_cls: 0.2528, acc: 91.9934, loss_bbox: 0.2978, loss: 0.6270, grad_norm: 1.5486
2021-05-12 09:46:35,193 - mmdet - INFO - Epoch [9][140/164]	lr: 2.000e-03, eta: 0:06:07, time: 0.703, data_time: 0.018, memory: 6896, loss_rpn_cls: 0.0415, loss_rpn_bbox: 0.0548, loss_cls: 0.2775, acc: 90.9998, loss_bbox: 0.3389, loss: 0.7127, grad_norm: 1.6968
2021-05-12 09:46:42,189 - mmdet - INFO - Epoch [9][150/164]	lr: 2.000e-03, eta: 0:06:00, time: 0.700, data_time: 0.018, memory: 6896, loss_rpn_cls: 0.0378, loss_rpn_bbox: 0.0502, loss_cls: 0.2536, acc: 91.8762, loss_bbox: 0.2775, loss: 0.6191, grad_norm: 1.5792
2021-05-12 09:46:49,204 - mmdet - INFO - Epoch [9][160/164]	lr: 2.000e-03, eta: 0:05:52, time: 0.701, data_time: 0.018, memory: 6896, loss_rpn_cls: 0.0316, loss_rpn_bbox: 0.0421, loss_cls: 0.2476, acc: 91.9836, loss_bbox: 0.2950, loss: 0.6163, grad_norm: 1.5140
2021-05-12 09:46:52,068 - mmdet - INFO - Saving checkpoint at 9 epo

train/loss_rpn_cls,0.02811
train/loss_rpn_bbox,0.04475
train/loss_cls,0.24672
train/acc,91.92993
train/loss_bbox,0.29949
train/loss,0.61907
train/grad_norm,1.58628
learning_rate,0.0002
momentum,0.9
_runtime,1782
_timestamp,1620813280


train/loss_rpn_cls,█▃▃▂▂▂▂▂▂▂▂▂▂▂▂▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/loss_rpn_bbox,█▅▆▄▇▄▅▆▄▄▅▄▃▃▅▂▄▃▃▃▃▃▃▃▃▃▃▃▁▃▂▁▁▁▂▁▁▂▂▂
train/loss_cls,█▆▆▅▆▆▆▆▅▆▆▆▅▄▅▄▅▄▄▄▄▄▃▄▃▃▄▃▂▂▂▁▂▁▂▁▂▁▁▁
train/acc,█▄▃▃▁▂▂▂▂▁▁▁▃▃▂▃▂▃▂▂▃▃▄▃▃▃▃▃▅▄▄▅▄▅▄▅▄▅▅▅
train/loss_bbox,▁▅▆▆█▇▇▇▇▇▇▇▆▆▇▅▆▆▆▆▆▆▅▆▅▆▆▆▅▆▅▄▅▄▅▅▅▅▅▅
train/loss,█▅▅▅▆▅▅▅▅▅▅▅▄▃▄▃▄▃▃▃▃▃▂▃▃▃▃▃▁▂▂▁▂▁▂▁▁▁▁▁
train/grad_norm,█▂▁▂▃▃▄▃▂▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▃▁▁▁▁▁▁▁▂▁▁▁▁▁
learning_rate,▁▂▃▃▄▅▅▆▇██████████████████▂▂▂▂▂▂▂▂▂▁▁▁▁
momentum,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
_runtime,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
_timestamp,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███


## Inference part

In [35]:
epoch = cfg.runner.max_epochs
cfg.model.train_cfg = None

# checkpoint path
checkpoint_path = os.path.join(cfg.work_dir, f'epoch_{epoch}.pth')

In [36]:
dataset = build_dataset(cfg.data.test)
data_loader = build_dataloader(
        dataset,
        samples_per_gpu=1,
        workers_per_gpu=cfg.data.workers_per_gpu,
        dist=False,
        shuffle=False)

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


In [37]:
model = build_detector(cfg.model, test_cfg=cfg.get('test_cfg'))
checkpoint = load_checkpoint(model, checkpoint_path, map_location='cpu')

model.CLASSES = dataset.CLASSES
model = MMDataParallel(model.cuda(), device_ids=[0])

2021-05-12 09:56:11,817 - mmdet - INFO - load model from: torchvision://resnet50
2021-05-12 09:56:11,818 - mmdet - INFO - Use load_from_torchvision loader
2021-05-12 09:56:12,042 - mmdet - WARNING - The model and loaded state dict do not match exactly

unexpected key in source state_dict: fc.weight, fc.bias

Use load_from_local loader


In [38]:
output = single_gpu_test(model, data_loader, show_score_thr=0.05)

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 837/837, 34.2 task/s, elapsed: 24s, ETA:     0s

In [39]:
prediction_strings = []
file_names = []
coco = COCO(cfg.data.test.ann_file)
imag_ids = coco.getImgIds()

class_num = 11
for i, out in enumerate(output):
    prediction_string = ''
    image_info = coco.loadImgs(coco.getImgIds(imgIds=i))[0]
    for j in range(class_num):
        for o in out[j]:
            prediction_string += str(j) + ' ' + str(o[4]) + ' ' + str(o[0]) + ' ' + str(o[1]) + ' ' + str(
                o[2]) + ' ' + str(o[3]) + ' '
        
    prediction_strings.append(prediction_string)
    file_names.append(image_info['file_name'])


submission = pd.DataFrame()
submission['PredictionString'] = prediction_strings
submission['image_id'] = file_names
submission.to_csv(os.path.join(cfg.work_dir, 'submission_'+config_file_name+'f_{epoch}.csv'), index=None)
submission.head()

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!


,PredictionString,image_id
0,1 0.27591115 165.25833 74.776306 260.0695 186....,batch_01_vt/0021.jpg
1,1 0.087882824 36.40812 0.0 150.26389 275.60645...,batch_01_vt/0028.jpg
2,1 0.42258692 122.51999 159.63556 381.79993 341...,batch_01_vt/0031.jpg
3,1 0.053491384 0.32694817 218.53006 39.504967 3...,batch_01_vt/0032.jpg
4,1 0.7363675 393.93417 280.24387 460.42484 346....,batch_01_vt/0070.jpg
